**INTRODUÇÃO DO PROJETO**

- Analise de Dados realizada com a linguagem de programação Python utilizada para entender o problema de "Churn de Clientes" da Telecom X, a empresa enfrenta um alto indice de cancelamentos e precisa entender os fatores que levam a essa perda de clientes ao longo do tempo.

**LIMPEZA E TRATAMENTO DOS DADOS**

-  **IMPORTAÇÃO DAS BIBLIOTECAS**

In [21]:
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px

**IMPORTAÇÃO DOS DADOS UTILIZADOS**

In [2]:
url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json'
df = pd.read_json(url)  # Atribui á uma variável os dados importados
df.head()  # Visualizar os dados

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


**TRANSFORMAÇÃO E NORMALIZAÇÃO**

In [3]:
df = requests.get(url) 
resultado = json.loads(df.text)

df = pd.json_normalize(resultado, sep='/') #normalize os dados e separe por uma /
df.head()


,customerID,Churn,customer/gender,customer/SeniorCitizen,customer/Partner,customer/Dependents,customer/tenure,phone/PhoneService,phone/MultipleLines,internet/InternetService,...,internet/OnlineBackup,internet/DeviceProtection,internet/TechSupport,internet/StreamingTV,internet/StreamingMovies,account/Contract,account/PaperlessBilling,account/PaymentMethod,account/Charges/Monthly,account/Charges/Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


**TRADUÇÃO DAS COLUNAS**

In [6]:
# dicionário de tradução
colunas_traduzidas = {
    'customerID': 'id_Cliente',
    'Churn': 'cancelamento',
    'customer/gender': 'genero',
    'customer/SeniorCitizen': 'idoso',
    'customer/Partner': 'possui_companheiro',
    'customer/Dependents': 'possui_dependentes',
    'customer/tenure': 'meses_de_contrato',
    'phone/PhoneService': 'telefone_ativo',
    'phone/MultipleLines': 'multiplas_linhas',
    'internet/InternetService': 'tipo_internet',
    'internet/OnlineSecurity': 'seguranca_online',
    'internet/OnlineBackup': 'backup_online',
    'internet/DeviceProtection': 'protecao_dispositivo',
    'internet/TechSupport': 'suporte_tecnico',
    'internet/StreamingTV': 'streaming_tv',
    'internet/StreamingMovies': 'streaming_filmes',
    'account/Contract': 'tipo_contrato',
    'account/PaperlessBilling': 'fatura_digital',
    'account/PaymentMethod': 'forma_pagamento',
    'account/Charges/Monthly': 'gasto_mensal',
    'account/Charges/Total': 'gasto_total',
    'tempo_estimado_meses': 'meses_estimados',
    'Conta_Diarias': 'conta_diaria'
}

# renomeando as colunas
df.rename(columns=colunas_traduzidas, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id_Cliente            7267 non-null   object 
 1   cancelamento          7267 non-null   object 
 2   genero                7267 non-null   object 
 3   idoso                 7267 non-null   int64  
 4   possui_companheiro    7267 non-null   object 
 5   possui_dependentes    7267 non-null   object 
 6   meses_de_contrato     7267 non-null   int64  
 7   telefone_ativo        7267 non-null   object 
 8   multiplas_linhas      7267 non-null   object 
 9   tipo_internet         7267 non-null   object 
 10  seguranca_online      7267 non-null   object 
 11  backup_online         7267 non-null   object 
 12  protecao_dispositivo  7267 non-null   object 
 13  suporte_tecnico       7267 non-null   object 
 14  streaming_tv          7267 non-null   object 
 15  streaming_filmes     

In [7]:
df.head()

,id_Cliente,cancelamento,genero,idoso,possui_companheiro,possui_dependentes,meses_de_contrato,telefone_ativo,multiplas_linhas,tipo_internet,...,backup_online,protecao_dispositivo,suporte_tecnico,streaming_tv,streaming_filmes,tipo_contrato,fatura_digital,forma_pagamento,gasto_mensal,gasto_total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [9]:
#alterarando o tipo de dados das coluna gasto_total
df['gasto_total'] = pd.to_numeric(df['gasto_total'], errors='coerce') #tratar os erros existentes nessa coluna
df[['gasto_total']] = df[['gasto_total']].astype(np.float64) # Converte as colunas para float64

df['idoso'] = df[['idoso']].astype(bool) #converter de int64 para booleano

**VERIFICAÇÃO DE ID_CLIENTES REPETIDOS**

In [10]:
# Remove linhas onde o 'ID_Cliente' é repetido, mantendo apenas a primeira ocorrência
df_sem_duplicatas = df.drop_duplicates(subset=['id_Cliente'], keep='first')
df_sem_duplicatas

,id_Cliente,cancelamento,genero,idoso,possui_companheiro,possui_dependentes,meses_de_contrato,telefone_ativo,multiplas_linhas,tipo_internet,...,backup_online,protecao_dispositivo,suporte_tecnico,streaming_tv,streaming_filmes,tipo_contrato,fatura_digital,forma_pagamento,gasto_mensal,gasto_total
0,0002-ORFBO,No,Female,False,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.30
1,0003-MKNFE,No,Male,False,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.40
2,0004-TLHLJ,Yes,Male,False,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,True,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,True,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,9987-LUTYD,No,Female,False,No,No,13,Yes,No,DSL,...,No,No,Yes,No,No,One year,No,Mailed check,55.15,742.90
7263,9992-RRAMN,Yes,Male,False,Yes,No,22,Yes,Yes,Fiber optic,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.70
7264,9992-UJOEL,No,Male,False,No,No,2,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75
7265,9993-LHIEB,No,Male,False,Yes,Yes,67,Yes,No,DSL,...,No,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65


**REMOÇÃO DE VALORES NULOS**

In [13]:
valores_nulos = df[df.isnull().any(axis=1)] #verificar se a dados nulos nas colunas
contagem_nulos = df.isnull().sum() #realizar a contagem desses dados nulos
contagem_nulos

id_Cliente              0
cancelamento            0
genero                  0
idoso                   0
possui_companheiro      0
possui_dependentes      0
meses_de_contrato       0
telefone_ativo          0
multiplas_linhas        0
tipo_internet           0
seguranca_online        0
backup_online           0
protecao_dispositivo    0
suporte_tecnico         0
streaming_tv            0
streaming_filmes        0
tipo_contrato           0
fatura_digital          0
forma_pagamento         0
gasto_mensal            0
gasto_total             0
dtype: int64

In [14]:
df = df[df['gasto_total'].notnull()] #função para remover valores nulos da coluna gasto total / só manter os valores não nulos
contagem_nulos

id_Cliente              0
cancelamento            0
genero                  0
idoso                   0
possui_companheiro      0
possui_dependentes      0
meses_de_contrato       0
telefone_ativo          0
multiplas_linhas        0
tipo_internet           0
seguranca_online        0
backup_online           0
protecao_dispositivo    0
suporte_tecnico         0
streaming_tv            0
streaming_filmes        0
tipo_contrato           0
fatura_digital          0
forma_pagamento         0
gasto_mensal            0
gasto_total             0
dtype: int64

**ENCONTRAR DADOS EM BRANCO**

In [17]:
#verificando se há dados em branco
df.apply(lambda x: x.astype(str).str.strip() == '').sum()

id_Cliente              0
cancelamento            0
genero                  0
idoso                   0
possui_companheiro      0
possui_dependentes      0
meses_de_contrato       0
telefone_ativo          0
multiplas_linhas        0
tipo_internet           0
seguranca_online        0
backup_online           0
protecao_dispositivo    0
suporte_tecnico         0
streaming_tv            0
streaming_filmes        0
tipo_contrato           0
fatura_digital          0
forma_pagamento         0
gasto_mensal            0
gasto_total             0
dtype: int64

In [18]:
#eliminar aonde os dados estão em branco
df = df[df['cancelamento'].str.strip() != '']
print("Número de linhas depois de eliminar as linhas vazias  'Cancelamento':", len(df))

Número de linhas depois de eliminar as linhas vazias  'Cancelamento': 7032


**CALCULAR O GASTO DIARIO PELOS CLIENTES**

In [19]:
df = df.drop('gasto_diario', errors='ignore') 
df['gasto_diario'] = ((df['gasto_total'] / df['meses_de_contrato']) / 30).round(2) #calcule o gasto diario dividindo o gasto total pelo tempo de contrato

#inserindo a coluna no data frame
df.insert(19, 'gasto_diario', df.pop('gasto_diario'))

In [20]:
df.head()

,id_Cliente,cancelamento,genero,idoso,possui_companheiro,possui_dependentes,meses_de_contrato,telefone_ativo,multiplas_linhas,tipo_internet,...,protecao_dispositivo,suporte_tecnico,streaming_tv,streaming_filmes,tipo_contrato,fatura_digital,forma_pagamento,gasto_diario,gasto_mensal,gasto_total
0,0002-ORFBO,No,Female,False,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,2.20,65.6,593.30
1,0003-MKNFE,No,Male,False,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,2.01,59.9,542.40
2,0004-TLHLJ,Yes,Male,False,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,2.34,73.9,280.85
3,0011-IGKFF,Yes,Male,True,Yes,No,13,Yes,No,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,3.17,98.0,1237.85
4,0013-EXCHZ,Yes,Female,True,Yes,No,3,Yes,No,Fiber optic,...,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,2.97,83.9,267.40


In [ ]:
df.describe() #resumo dos valores do data frame

,meses_de_contrato,gasto_diario,gasto_mensal,gasto_total
count,7032.000000,7032.000000,7032.000000,7032.000000
mean,32.421786,2.159879,64.798208,2283.300441
std,24.545260,1.006217,30.085974,2266.771362
min,1.000000,0.460000,18.250000,18.800000
25%,9.000000,1.207500,35.587500,401.450000
50%,29.000000,2.350000,70.350000,1397.475000
75%,55.000000,3.010000,89.862500,3794.737500
max,72.000000,4.050000,118.750000,8684.800000


**ANÁLISE EXPLORATORIA DOS DADOS**

In [23]:
#variavel para saber a quantidade de clientes que cancelaram
contagem_cancelamento = df['cancelamento'].value_counts().reset_index()
contagem_cancelamento.columns = ['cancelamento', 'quantidade']

#geração do gráfico de pizza
fig = px.pie(contagem_cancelamento, names='cancelamento', values='quantidade', title='Visão Geral de Churn da Telecom X')
fig.update_layout(
    width =900,
    height= 400,
    font_size=14,
    font_family='Arial',
    title_font_color='black',
    title_x=0.5
)
fig.show()

In [32]:
#Taxa de Cancelamento = (Número de Pessoas que Cancelaram / Número TOTAL de Pessoas naquele mês) * 100

df_cancelamento = (df['cancelamento'] == 'Yes').sum()
print(f"Total de Cancelamentos (Yes): {df_cancelamento}")

df_cancelamento = (df['cancelamento'] == 'No').sum()
print(f"Total de Cancelamentos (No): {df_cancelamento}")

Total de Cancelamentos (Yes): 1869
Total de Cancelamentos (No): 5163


In [24]:
contagem_genero = df['genero'].value_counts().reset_index() #realizando a contagem de homem e mulher
contagem_genero.columns = ['genero', 'quantidade']

# Criando o gráfico de pizza
fig = px.pie(contagem_genero, names='genero', values='quantidade',  title='Divisão de Clientes da Telecom X por Gênero')
fig.update_layout(
    width =900,
    height= 400,
    font_size=14,
    font_family='Arial',
    title_font_color='black',
    title_x=0.5
)
fig.show()

In [25]:
#correlação entre genero e cancelamento
fig = px.histogram(df, x='genero', text_auto= True, color='cancelamento', barmode = 'group', title='Relação entre Gênero e Cancelamento',
        color_discrete_map={ #função para decidir as cores de cada valor dentro do nosso gráfico
        'Yes': 'blue',
        'No': 'red'
    }
)

fig.update_layout(
    width=1000,
    height=600,
    title_x=0.5,
    yaxis_title='Quantidade',
    font_family='Arial',
    title_font_color='black',
    xaxis_title='Gênero',
    font_size=14
)


In [27]:
#pessoas com mais de 65 anos
fig = px.histogram(df, x='idoso', text_auto= True, color='cancelamento', barmode = 'group', title='Pessoas Idosas e Relacão com o Cancelamento', 
    color_discrete_map={ #função para decidir as cores de cada valor dentro do nosso gráfico
        'Yes': 'blue',
        'No': 'red'
    }
)
fig.update_layout(
    width=1000,
    height=600,
    title_x=0.5,
    yaxis_title='Quantidade',
    font_family='Arial',
    title_font_color='black',
    xaxis_title='Idosos +65',
    font_size=14
)

In [28]:
fig = px.histogram(df, x='meses_de_contrato', text_auto= True, color='cancelamento', barmode = 'group', title='Relação entre Meses de Contrato e Cancelamento', 
    color_discrete_map={ #função para decidir as cores de cada valor dentro do nosso gráfico
        'Yes': 'blue',
        'No': 'red'
    }
)
fig.update_layout(
    width=1000,
    height=700,
    title_x=0.5,
    yaxis_title='Quantidade de Cancelamentos',
    font_family='Arial',
    title_font_color='black',
    xaxis_title='Meses de Contrato',
    font_size=14
)


fig.show()

INSIGHT: 
- Quanto **maior o tempo de contrato** menor é o numero de cancelamentos 
- **quanto menor é o tempo de contrato** maior é o numero de cancelamentos

In [29]:
df['meses_de_contrato'].describe() #vião geral da coluna meses de contrato

count    7032.000000
mean       32.421786
std        24.545260
min         1.000000
25%         9.000000
50%        29.000000
75%        55.000000
max        72.000000
Name: meses_de_contrato, dtype: float64

In [33]:
#eu quero saber quantas pessoas realizaram o cancelamento de acordo com os meses de contrato
df_meses = df[df['cancelamento'] == 'Yes'].groupby('meses_de_contrato').size()
df_meses

meses_de_contrato
1     380
2     123
3      94
4      83
5      64
     ... 
68      9
69      8
70     11
71      6
72      6
Length: 72, dtype: int64

In [43]:
#agrupar por meses de contrato e cancelamento
df_agrupado = df.groupby(['meses_de_contrato', 'cancelamento']).size().unstack(fill_value=0)

#calcular a taxa de cancelamento por tempo de contrato
df_agrupado['taxa_de_cancelamento'] = (df_agrupado.get('Yes', 0) / df_agrupado.sum(axis=1)) * 100

#exibir o resultado já formatado
df_agrupado

cancelamento,No,Yes,taxa_de_cancelamento
meses_de_contrato,,,
1,233,380,61.990212
2,115,123,51.680672
3,106,94,47.000000
4,93,83,47.159091
5,69,64,48.120301
...,...,...,...
68,91,9,9.000000
69,87,8,8.421053
70,108,11,9.243697


In [44]:
#gráfico de linhas

fig = px.line(
    df_agrupado.reset_index(),
    x='meses_de_contrato',
    y='taxa_de_cancelamento',
    title='Taxa de Cancelamento por meses de Contrato',
    markers=True
)

fig.update_layout(
    width=900,
    height=700,
    title_x=0.5,
    xaxis_title='Meses de Contrato',
    yaxis_title='Taxa de Cancelamento (%)',
    font_family='Arial',
    title_font_color='black',
    font_size=14
)
fig.show()

**INSIGHT**
- QUANTO **MAIOR O TEMPO DE CONTRATO** MENOR A TAXA DE CANCELAMENTO
E QUANTO **MENOR O TEMPO DE CONTRATO** MAIOR A TAXA DE CANCELAMENTO

In [45]:
gasto_por_contrato = df.groupby('forma_pagamento')['gasto_total'].size() #agrupa as formas de pagamento e conta cada um
gasto_por_contrato

forma_pagamento
Bank transfer (automatic)    1542
Credit card (automatic)      1521
Electronic check             2365
Mailed check                 1604
Name: gasto_total, dtype: int64

In [46]:
# Agrupa e conta quantos clientes por forma de pagamento em ordem crescente
clientes_forma_pagamento = df.groupby('forma_pagamento').size().reset_index(name='quantidade')
clientes_forma_pagamento = clientes_forma_pagamento.sort_values(by='quantidade', ascending=True)

# Cria o gráfico de barras horizontais com rótulo de dados
fig = px.bar(
    clientes_forma_pagamento,
    x='quantidade',
    y='forma_pagamento',
    orientation='h',
    text='quantidade',
    title='Quantidade de Clientes por Forma de Pagamento',
    
)

fig.update_layout(
    width=1000,
    height=550,
    font_family='Arial',
    font_size=14,
    title_font_color='black',
    title_x=0.5,
    xaxis_title='Quantidade',
    yaxis_title='Forma de Pagamento'
)

fig.show()

In [ ]:
# Agrupa por forma de pagamento e cancelamento, conta os clientes
df_group = df.groupby(['forma_pagamento', 'cancelamento']).size().reset_index(name='quantidade')

# Calcula o total de clientes por forma de pagamento
df_group['total'] = df_group.groupby('forma_pagamento')['quantidade'].transform('sum')

# Calcula a porcentagem de cancelamento para cada grupo
df_group['porcentagem'] = (df_group['quantidade'] / df_group['total']) * 100

df_group

,forma_pagamento,cancelamento,quantidade,total,porcentagem
0,Bank transfer (automatic),No,1284,1542,83.268482
1,Bank transfer (automatic),Yes,258,1542,16.731518
2,Credit card (automatic),No,1289,1521,84.746877
3,Credit card (automatic),Yes,232,1521,15.253123
4,Electronic check,No,1294,2365,54.714588
5,Electronic check,Yes,1071,2365,45.285412
6,Mailed check,No,1296,1604,80.798005
7,Mailed check,Yes,308,1604,19.201995


In [ ]:
fig = px.bar(df_group, x='forma_pagamento', y='porcentagem', color='cancelamento', barmode='group', text='porcentagem', title='Taxa de Cancelamento (%) por Meio de Pagamento',
    color_discrete_map={ #função para decidir as cores de cada valor dentro do nosso gráfico
        'Yes': 'blue',
        'No': 'red'
    }
)

fig.update_traces(texttemplate='%{text:.2f}%') #formatação para porcentagem no rotulo de dados das barras do gráfico
fig.update_layout(
    width=1000,
    height=600,
    font_family='Arial',
    font_size=14,
    title_font_color='black',
    title_x=0.5,
    xaxis_title='Forma de Pagamento',
    yaxis_title='Taxa (%)'
)
fig.show()

**INSIGHT**
- PODEMOS OBSERVAR QUE O MAIOR NUMERO DE CANCELAMENTOS VEIO AO ESCOLHER ELETRONIC CHECK COMO OPÇÃO DE PAGAMENTO(ENTENDER UM POUCO MAIS SOBRE ESSE TIPO DE ESCOLHA DE PAGAMENTO)